In [1]:
#!conda install -c conda-forge googlemaps
#!pip install googlemaps
#!conda install -c conda-forge folium=0.5.0 --yes
#!pip install folium

In [2]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

import folium # Map plotting library

from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe
import json

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

# Import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import re

### Get all location in Delhi

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi').text 
soup = BeautifulSoup(source, 'lxml')

In [4]:
neighborhood=[]
for i in soup.find_all("h2"):
    for k in i.next:
        flag=0
        if "".join(list(k)) in ['See also','References','External links']:
            flag=1
            break
    if flag==1:
        continue
    for j in i.next_siblings:
        if j.name=='ul':
            text=j.text
            delimiters = ",", ".", "[",";"
            regexPattern = '|'.join(map(re.escape, delimiters))
            for m in text.split('\n'):
                neighborhood.append(re.split(regexPattern,m)[0])
            break

In [5]:
df = pd.DataFrame()
df['Location']=neighborhood

In [6]:
df.head()

,Location
0,Adarsh Nagar
1,Ashok Vihar
2,Begum Pur
3,Karala
4,Model Town


### Get the coordinates of each loaction

In [7]:
# define the rows to get the data
df['Latitude']=None
df['Longitude']=None

In [8]:
def coordinate_extract(soup):
    coor=[]
    search = soup.find_all(class_='BNeawe s3v9rd AP7Wnd')
    for i in search:
        try:
            c=float(i.text)
            coor.append(c)
        except:
            continue
    if len(coor)<2:
        try:
            for i in map(lambda x: re.findall(r'[0-9]{2}\.[0-9]{6}',x.text),search):
                coor.extend(list(map(lambda x: float(x), i)))
            if len(coor)<2: 
                raise exception('')
        except:
            for i in map(lambda x: re.findall(r'[0-9]{2}\.[0-9]{2,6}',x.text),search):
                coor.extend(list(map(lambda x: float(x), i)))
        finally:
            coor=coor[:2]
            
    return coor[0],coor[1]

In [9]:
for i in df['Location']:
    
    name='+'.join(i.split())
    
    url="https://www.google.com/search?q=delhi {} coordinates".format(name) # Google search at each iterations
    source_code = requests.get(url).text
    soup = BeautifulSoup(source_code, 'lxml')
    
    df.loc[df['Location']==i,['Latitude','Longitude']] = coordinate_extract(soup) # Updating the table

In [10]:
df.head(3)

,Location,Latitude,Longitude
0,Adarsh Nagar,28.7196,77.1741
1,Ashok Vihar,28.6904,77.1762
2,Begum Pur,28.7227,77.0627


### Map the locations using folium

In [11]:
# Delhi latitude and longitude using Google search
del_lat = 28.7041
del_lng = 77.1025

In [12]:
# Creates map of Delhi using latitude and longitude values
map_delhi = folium.Map(location=[del_lat, del_lng], zoom_start=11)

# Add markers to map
for lat, lng, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Location']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  

In [13]:
map_delhi

### Use foursquare api to get venue data

**Credentials**

CLIENT_ID, CLIENT_SECRET, VERSION

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
explore_df_list = []

for i, nbd_name in enumerate(df['Location']):  
    
    try :
        # Getting the data of Neighbours
        nbd_name = df.loc[i, 'Location']
        nbd_lat = df.loc[i, 'Latitude']
        nbd_lng = df.loc[i, 'Longitude']

        radius = 1000 # Setting the radius as 1000 metres
        LIMIT = 50 # Getting the top 30 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nbd_name, nbd_lat, nbd_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass

In [17]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Location', 'Location Latitude', 'Location Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']

In [18]:
explore_df.head(10)

,Location,Location Latitude,Location Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Adarsh Nagar,28.719646,77.174065,Pahalwan Vaishno Dhaba,North Indian Restaurant,28.715881,77.173565
1,Adarsh Nagar,28.719646,77.174065,Adarsh Nagar Metro Station,Light Rail Station,28.716598,77.170436
2,Adarsh Nagar,28.719646,77.174065,Vishyavidyalaya Metro Station@Entry gate #1 n ...,Train Station,28.715596,77.170981
3,Adarsh Nagar,28.719646,77.174065,Axis Bank ATM,ATM,28.722080,77.168740
4,Adarsh Nagar,28.719646,77.174065,Woodland Original,Clothing Store,28.716670,77.183330
5,Ashok Vihar,28.690351,77.176239,Bellagio,Asian Restaurant,28.696361,77.180021
6,Ashok Vihar,28.690351,77.176239,Subway,Sandwich Place,28.696321,77.179983
7,Ashok Vihar,28.690351,77.176239,Domino's Pizza,Pizza Place,28.693000,77.177000
8,Ashok Vihar,28.690351,77.176239,Rahul Egg Corner,Snack Place,28.688240,77.168599
9,Ashok Vihar,28.690351,77.176239,Subway.,Sandwich Place,28.695571,77.171964


### Seprate the data with accordance of place where we can get food 

In [19]:
#all the places where one can get some food

meals=['Restaurant','Place','Food','Joint','Breakfast','Hotel','Coffee','Donut','Dessert','Bakery','Diner','Café','Ice Cream','Cafeteria']

data=explore_df[(explore_df['Venue Category']).apply(lambda x: True if any(item in x.split() for item in meals) else False)==True ]

data.reset_index(inplace=True,drop=True)

data.head(10)

,Location,Location Latitude,Location Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Adarsh Nagar,28.719646,77.174065,Pahalwan Vaishno Dhaba,North Indian Restaurant,28.715881,77.173565
1,Ashok Vihar,28.690351,77.176239,Bellagio,Asian Restaurant,28.696361,77.180021
2,Ashok Vihar,28.690351,77.176239,Subway,Sandwich Place,28.696321,77.179983
3,Ashok Vihar,28.690351,77.176239,Domino's Pizza,Pizza Place,28.693000,77.177000
4,Ashok Vihar,28.690351,77.176239,Rahul Egg Corner,Snack Place,28.688240,77.168599
5,Ashok Vihar,28.690351,77.176239,Subway.,Sandwich Place,28.695571,77.171964
6,Ashok Vihar,28.690351,77.176239,Sagar Ratna | सागर रतना,South Indian Restaurant,28.693381,77.177977
7,Ashok Vihar,28.690351,77.176239,Invitation Restaurant,Indian Restaurant,28.696182,77.179287
8,Ashok Vihar,28.690351,77.176239,Cafe Coffee Day,Coffee Shop,28.696103,77.179944
9,Ashok Vihar,28.690351,77.176239,Giani,Dessert Shop,28.692699,77.171194


In [20]:
map_delhi = folium.Map(location=[del_lat, del_lng], zoom_start=11)

# Add markers to map
for lat, lng, Location in zip(data['Venue Latitude'], data['Venue Longitude'], data['Venue Name']):
    label = '{}'.format(Location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi) 

In [21]:
map_delhi

In [22]:
# Example coordinate
a1=28.6304
a2=77.2177

In [23]:
from geopy.distance import distance
place={}
for i in zip(data['Venue Name'],data['Venue Latitude'],data['Venue Longitude']):
    coor=(i[1],i[2])
    k=distance(coor,(a1,a2))
    if k<=2.000000000000000:
        place[i[0]]=[k,i[1],i[2]]
        
places={k: v for k, v in sorted(place.items(), key=lambda item: item[1][0])}

In [24]:
nearby_venue_latitude=[]
nearby_venue_longitude=[]

for i in places.items():
    nearby_venue_latitude.append(i[1][1])
    nearby_venue_longitude.append(i[1][2])

In [25]:
final=pd.DataFrame()
final['Name']=places.keys()
final['Latitude']=nearby_venue_latitude
final['Longitude']=nearby_venue_longitude

In [26]:
final.head()

,Name,Latitude,Longitude
0,Pind Balluchi,28.630318,77.217600
1,Jain Chawal Wale,28.630052,77.217649
2,Starbucks,28.632011,77.217731
3,Dunkin',28.630182,77.219622
4,Nando's,28.630947,77.219721


In [27]:
map_nearby = folium.Map(location=[a1, a2], zoom_start=14)

folium.Marker([a1,a2]).add_to(map_nearby)

for lat, lng, Location in zip(final['Latitude'], final['Longitude'], final['Name']):
    label = '{}'.format(Location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nearby) 

In [28]:
map_nearby